In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [23]:
train = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/training_data.csv")
test = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/testing_data.csv")


In [24]:
user = pd.read_excel("/content/User.xlsx")

In [25]:
user

,UserID,Timestamp,Bạn đến từ vùng miền nào,Tuổi của bạn,Giới tính của ban,Bạn thuộc nhóm cân nặng nào? Chỉ số chiều cao và cân nặng để có thể đưa ra những khuyến nghị món ăn phù hợp với thể trạng của bạn nhất.,Bạn thuộc nhóm chiều cao nào?,Bạn thích nhóm món ăn gì
0,U236,2021-12-22 20:00:04.176,Miền Bắc,22,Nam,45-55 kg,1m55 - 1m65,"Món nước (Bún, Phở), Món khô (Xôi, Bánh mì), M..."
1,U237,2021-12-22 20:07:31.010,Miền Trung,23,Nam,45-55 kg,1m55 - 1m65,"Món nước (Bún, Phở), Món nướng"
2,U238,2021-12-22 20:18:08.981,Miền Nam,21,Nam,55-65 kg,1m65 - 1m75,"Món nước (Bún, Phở), Món khô (Xôi, Bánh mì), M..."
3,U239,2021-12-22 20:23:01.283,Miền Nam,21,Nam,55-65 kg,1m55 - 1m65,"Món nước (Bún, Phở), Món khô (Xôi, Bánh mì), M..."
4,U240,2021-12-22 22:16:22.478,Miền Bắc,21,Nam,Trên 65 kg,Trên 1m75,"Món nước (Bún, Phở)"


In [26]:
favorite = []
for i in user['Bạn thích nhóm món ăn gì'].values:

  i = re.sub('( \(Xôi, Bánh mì\))', '', i)
  i = re.sub('( \(Bún, Phở\))', '', i)
  favorite.append(i)

user['Bạn thích nhóm món ăn gì'] = favorite

In [27]:
def split(row):
  return row.split(", ")

user["list_favorite"] = user['Bạn thích nhóm món ăn gì'].apply(split)

In [28]:
def create_new_columns(df):
  df1 = df.copy()
  df1['is_tat_ca'] = 0
  df1['is_mon_nuoc'] = 0
  df1['is_mon_kho']  = 0
  df1['is_mon_nuong'] = 0
  df1['is_mon_cay'] = 0

  for i in range(df1.shape[0]):
    if "Món nước" in df1.loc[i, "list_favorite"]:
      df1.loc[i, 'is_mon_nuoc'] = 1
    if "Món khô" in df1.loc[i, "list_favorite"]:
      df1.loc[i, 'is_mon_kho'] = 1
    if "Món nướng" in df1.loc[i, "list_favorite"]:
      df1.loc[i,'is_mon_nuong'] = 1
    if "Món cay" in df1.loc[i, "list_favorite"]:
      df1.loc[i,'is_mon_cay'] = 1
    if ("Tất cả" or 'tất cả' or 'Thích hết') in df1.loc[i, "list_favorite"]:
      df1.loc[i,'is_tat_ca'] = 1
      df1.loc[i,'is_mon_nuoc'] = 0
      df1.loc[i,'is_mon_kho'] = 0
      df1.loc[i,'is_mon_nuong'] = 0
      df1.loc[i,'is_mon_cay'] = 0
  return df1

user_favorite = create_new_columns(user)

In [29]:
user_favorite

,UserID,Timestamp,Bạn đến từ vùng miền nào,Tuổi của bạn,Giới tính của ban,Bạn thuộc nhóm cân nặng nào? Chỉ số chiều cao và cân nặng để có thể đưa ra những khuyến nghị món ăn phù hợp với thể trạng của bạn nhất.,Bạn thuộc nhóm chiều cao nào?,Bạn thích nhóm món ăn gì,list_favorite,is_tat_ca,is_mon_nuoc,is_mon_kho,is_mon_nuong,is_mon_cay
0,U236,2021-12-22 20:00:04.176,Miền Bắc,22,Nam,45-55 kg,1m55 - 1m65,"Món nước, Món khô, Món nướng","[Món nước, Món khô, Món nướng]",0,1,1,1,0
1,U237,2021-12-22 20:07:31.010,Miền Trung,23,Nam,45-55 kg,1m55 - 1m65,"Món nước, Món nướng","[Món nước, Món nướng]",0,1,0,1,0
2,U238,2021-12-22 20:18:08.981,Miền Nam,21,Nam,55-65 kg,1m65 - 1m75,"Món nước, Món khô, Món cay, Món nướng, Tất cả","[Món nước, Món khô, Món cay, Món nướng, Tất cả]",1,0,0,0,0
3,U239,2021-12-22 20:23:01.283,Miền Nam,21,Nam,55-65 kg,1m55 - 1m65,"Món nước, Món khô, Món nướng","[Món nước, Món khô, Món nướng]",0,1,1,1,0
4,U240,2021-12-22 22:16:22.478,Miền Bắc,21,Nam,Trên 65 kg,Trên 1m75,Món nước,[Món nước],0,1,0,0,0


In [30]:
user_favorite.columns

Index(['UserID', 'Timestamp', 'Bạn đến từ vùng miền nào', 'Tuổi của bạn',
       'Giới tính của ban',
       'Bạn thuộc nhóm cân nặng nào? Chỉ số chiều cao và cân nặng để có thể đưa ra những khuyến nghị món ăn phù hợp với thể trạng của bạn nhất. ',
       'Bạn thuộc nhóm chiều cao nào?', 'Bạn thích nhóm món ăn gì',
       'list_favorite', 'is_tat_ca', 'is_mon_nuoc', 'is_mon_kho',
       'is_mon_nuong', 'is_mon_cay'],
      dtype='object')

In [31]:
categorical = user_favorite.loc[:, ['Bạn đến từ vùng miền nào', "Giới tính của ban"]]
categorical

,Bạn đến từ vùng miền nào,Giới tính của ban
0,Miền Bắc,Nam
1,Miền Trung,Nam
2,Miền Nam,Nam
3,Miền Nam,Nam
4,Miền Bắc,Nam


In [32]:
raw_user = pd.concat([user_favorite, pd.get_dummies(categorical)], axis=1)

In [33]:
raw_user.columns

Index(['UserID', 'Timestamp', 'Bạn đến từ vùng miền nào', 'Tuổi của bạn',
       'Giới tính của ban',
       'Bạn thuộc nhóm cân nặng nào? Chỉ số chiều cao và cân nặng để có thể đưa ra những khuyến nghị món ăn phù hợp với thể trạng của bạn nhất. ',
       'Bạn thuộc nhóm chiều cao nào?', 'Bạn thích nhóm món ăn gì',
       'list_favorite', 'is_tat_ca', 'is_mon_nuoc', 'is_mon_kho',
       'is_mon_nuong', 'is_mon_cay', 'Bạn đến từ vùng miền nào_Miền Bắc',
       'Bạn đến từ vùng miền nào_Miền Nam',
       'Bạn đến từ vùng miền nào_Miền Trung', 'Giới tính của ban_Nam'],
      dtype='object')

In [34]:
a = 'Bạn thuộc nhóm cân nặng nào? Chỉ số chiều cao và cân nặng để có thể đưa ra những khuyến nghị món ăn phù hợp với thể trạng của bạn nhất. '
raw_user[a].replace("Dưới 35 kg", 1, inplace = True)
raw_user[a].replace("35-45 kg", 2, inplace = True)
raw_user[a].replace("45-55 kg", 3, inplace = True)
raw_user[a].replace("55-65 kg", 4, inplace = True)
raw_user[a].replace("Trên 65 kg", 5, inplace = True)
raw_user[a].fillna(0, inplace = True)

In [35]:
b = 'Bạn thuộc nhóm chiều cao nào?'
raw_user[b].replace("Dưới 1m45", 1, inplace = True)
raw_user[b].replace("1m45 -1m55", 2, inplace = True)
raw_user[b].replace("1m55 - 1m65", 3, inplace = True)
raw_user[b].replace("1m65 - 1m75", 4, inplace = True)
raw_user[b].replace("Trên 1m75", 5, inplace = True)
raw_user[b].fillna(0, inplace = True)

In [36]:
raw_user.columns = ['UserID', 'Timestamp', 'Vùng miền', 'Tuổi',
       'Giới tính',
       'Cân nặng',
       'Chiều cao', 'Yêu thích',
       'list_favorite', 'is_tat_ca', 'is_mon_nuoc', 'is_mon_kho',
       'is_mon_nuong', 'is_mon_cay', 'Vùng miền_Miền Bắc',
       'Vùng miền_Miền Nam',
       'Vùng miền_Miền Trung', 'Giới tính _Nam']

In [39]:
raw_user['Giới tính _Nữ'] = 0
raw_user['Giới tính _Khác'] = 0

In [40]:
raw_user1 = raw_user[["UserID", "Tuổi", 'Cân nặng', 'Chiều cao', 
                      'is_tat_ca', 'is_mon_nuoc', 'is_mon_kho',
                      'is_mon_nuong', 'is_mon_cay', 'Vùng miền_Miền Bắc',
                      'Vùng miền_Miền Nam', 'Vùng miền_Miền Trung', 'Giới tính _Khác',
                      'Giới tính _Nam', 'Giới tính _Nữ']]

In [43]:
raw_user1['Tuổi'].isnull().sum()

0

In [44]:
raw_user1['Tuổi'].replace("18 tuổi", 18, inplace = True)
raw_user1['Tuổi'].fillna(raw_user1['Tuổi'].median(), inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [45]:
raw_user1.columns

Index(['UserID', 'Tuổi', 'Cân nặng', 'Chiều cao', 'is_tat_ca', 'is_mon_nuoc',
       'is_mon_kho', 'is_mon_nuong', 'is_mon_cay', 'Vùng miền_Miền Bắc',
       'Vùng miền_Miền Nam', 'Vùng miền_Miền Trung', 'Giới tính _Khác',
       'Giới tính _Nam', 'Giới tính _Nữ'],
      dtype='object')

In [46]:
columns_name = ['UserID', 'Age', 'Weight', 'Height', 'like_all',
       'like_mon_nuoc', 'like_mon_kho', 'like_mon_nuong', 'like_mon_cay',
       'is_north', 'is_south', 'is_middle',
       'gender_others', 'gender_male', 'gender_female']

In [47]:
raw_user1.columns = columns_name

In [48]:
raw_user1

,UserID,Age,Weight,Height,like_all,like_mon_nuoc,like_mon_kho,like_mon_nuong,like_mon_cay,is_north,is_south,is_middle,gender_others,gender_male,gender_female
0,U236,22,3,3,0,1,1,1,0,1,0,0,0,1,0
1,U237,23,3,3,0,1,0,1,0,0,0,1,0,1,0
2,U238,21,4,4,1,0,0,0,0,0,1,0,0,1,0
3,U239,21,4,3,0,1,1,1,0,0,1,0,0,1,0
4,U240,21,5,5,0,1,0,0,0,1,0,0,0,1,0


In [49]:
raw_user1.to_csv("/content/drive/MyDrive/Food Recommend Project/data/test_user.csv", index = None)

In [50]:
raw_user1

,UserID,Age,Weight,Height,like_all,like_mon_nuoc,like_mon_kho,like_mon_nuong,like_mon_cay,is_north,is_south,is_middle,gender_others,gender_male,gender_female
0,U236,22,3,3,0,1,1,1,0,1,0,0,0,1,0
1,U237,23,3,3,0,1,0,1,0,0,0,1,0,1,0
2,U238,21,4,4,1,0,0,0,0,0,1,0,0,1,0
3,U239,21,4,3,0,1,1,1,0,0,1,0,0,1,0
4,U240,21,5,5,0,1,0,0,0,1,0,0,0,1,0


# Food description

In [ ]:
df1 = pd.read_excel("/content/drive/MyDrive/Food Recommend Project/data/Food_description.xlsx")

In [ ]:
df1.iloc[0, 1]+" "+df1.iloc[0, 2]+" "+df1.iloc[0, 3]+" "+df1.iloc[0, 4]+" "+df1.iloc[0, 5]

'Xôi Miền Bắc, Miền Trung, Miền Nam Gạo nếp, đậu xanh, Vừng, Đậu phộng Món khô Xôi được làm từ gạo nếp hấp chín bằng hơi nước. Dùng chủ yếu để ăn sáng, giúp no lâu. Ăn kèm với lạc, vừng, dừa cho hương vị béo, thơm, ngọt. '